# Explore Transformers

We wil use the `transformers` package from `HuggingFace`, that provides a unified interface to a variety of Transormer nodels.

In [3]:
!pip3 install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 


In [4]:
from transformers import AutoModel, AutoTokenizer

Pretrained models can be downloaded directly from the HuggingFace repository.

We need also the Tokenizer for a given model, since each model may do it differently.

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Here we get the base BERT model `uncased`, i.i. where toknes are all lowercased.

In [6]:
model = AutoModel.from_pretrained('bert-base-uncased') 

In [7]:
wps_ids = tokenizer.encode("Hypatia was a mathematician")
wps_ids

[101, 1044, 22571, 10450, 2050, 2001, 1037, 13235, 102]

In [8]:
wordpieces = tokenizer.convert_ids_to_tokens(wps_ids)

See what they are:

In [9]:
wordpieces

['[CLS]', 'h', '##yp', '##ati', '##a', 'was', 'a', 'mathematician', '[SEP]']

Convert to tensor

In [10]:
import torch
wps_tensor = torch.tensor([wps_ids])

In [11]:
outputs = model(wps_tensor, output_hidden_states=True, output_attentions=True)

In [12]:
last_hidden_state = outputs.last_hidden_state
pooler_output = outputs.pooler_output
hidden_states = outputs.hidden_states
attentions = outputs.attentions

`last_hidden_state` is the sequence of hidden-states at the output of the last layer of the model.

shape: (batch_size, sequence_length, hidden_size)

In [13]:
last_hidden_state[0]

tensor([[-0.1904, -0.2317, -0.4896,  ..., -0.4366,  0.5167,  0.5166],
        [ 0.7274,  0.4021, -0.4051,  ..., -0.9010,  1.4341,  0.2526],
        [-0.2553, -0.1450, -0.4257,  ..., -0.8199,  1.1043,  0.2408],
        ...,
        [ 0.1278,  0.1691, -0.8889,  ..., -0.6259,  0.1597,  0.8265],
        [-0.8941, -0.1419, -0.6008,  ..., -0.1861,  0.8240,  0.4556],
        [ 0.7600,  0.0256, -0.5482,  ...,  0.2768, -0.5651, -0.2150]],
       grad_fn=<SelectBackward>)

In [14]:
last_hidden_state[0].shape

torch.Size([9, 768])

`pooler_output` Last layer hidden-state of the first token of the sequence (classification token) further processed by a Linear layer and a Tanh activation function. The Linear layer weights are trained from the next sentence prediction (classification) objective during pretraining.

shape: `(batch_size, hidden_size)`

In [15]:
len(pooler_output[0])

768

`hidden_states` is a Tuple of `torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer.

shape of each layer: `(batch_size, sequence_length, hidden_size)`

In [16]:
len(hidden_states)

13

Look at the last layer

In [17]:
hidden_states[-1][0]

tensor([[-0.1904, -0.2317, -0.4896,  ..., -0.4366,  0.5167,  0.5166],
        [ 0.7274,  0.4021, -0.4051,  ..., -0.9010,  1.4341,  0.2526],
        [-0.2553, -0.1450, -0.4257,  ..., -0.8199,  1.1043,  0.2408],
        ...,
        [ 0.1278,  0.1691, -0.8889,  ..., -0.6259,  0.1597,  0.8265],
        [-0.8941, -0.1419, -0.6008,  ..., -0.1861,  0.8240,  0.4556],
        [ 0.7600,  0.0256, -0.5482,  ...,  0.2768, -0.5651, -0.2150]],
       grad_fn=<SelectBackward>)

Look at the one for the first wordpiece (skipping (CLS]):

In [18]:
hidden_states[0][-1][1]

tensor([ 5.5931e-01, -2.9349e-01,  1.5142e-01, -6.6894e-01, -3.0138e-01,
        -8.1427e-01,  2.7488e-01, -2.3421e-01, -6.0991e-01, -5.4946e-01,
        -7.7725e-01, -1.2423e+00, -4.5414e-01,  4.8284e-01, -5.9605e-01,
        -7.7930e-01,  3.9271e-01, -8.5450e-02, -5.2417e-01,  3.6583e-01,
         5.5155e-01, -9.3273e-01,  6.2493e-01,  1.1886e+00,  8.0808e-01,
        -1.8684e-01, -9.5162e-01,  1.0439e+00, -5.9416e-01,  6.8520e-01,
        -3.5158e-01, -2.2820e-01,  6.8480e-02, -2.8891e-01,  2.8974e-01,
        -8.4148e-01, -1.7565e+00, -4.3860e-01, -2.6000e-01,  1.2688e-01,
         1.1053e+00,  7.0817e-01,  1.7711e-01,  8.7551e-02,  2.4324e-02,
         3.2612e-01,  1.2759e-01, -6.7901e-01,  4.2167e-01, -1.1967e+00,
        -4.2347e-01,  3.7393e-01, -8.0793e-01, -2.8902e-02, -7.6317e-01,
        -5.8712e-01,  7.4728e-02, -4.7688e-01,  4.4490e-01, -5.4128e-01,
        -7.2543e-02,  4.3697e-01, -3.0546e-02, -1.8312e-01,  7.0248e-01,
         3.3365e-01, -5.8335e-01,  1.0221e+00,  2.6

`attentions` Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

shape: `(batch_size, num_heads, sequence_length, sequence_length)`

In [19]:
attentions[0].shape

torch.Size([1, 12, 9, 9])

Look at the single head:

In [20]:
attentions[0][0].shape

torch.Size([12, 9, 9])

Attentions for last layer

In [21]:
attentions[0][0][-1].shape

torch.Size([9, 9])

In [22]:
attentions[0][0][-1]

tensor([[8.9986e-01, 8.9166e-03, 3.4554e-03, 8.5585e-04, 2.9731e-03, 4.7226e-03,
         3.5215e-02, 4.7056e-03, 3.9298e-02],
        [1.2391e-01, 8.4317e-02, 3.2262e-01, 1.5241e-01, 4.7131e-02, 6.6522e-02,
         5.5258e-02, 2.4437e-02, 1.2340e-01],
        [2.3238e-03, 9.2049e-01, 2.0366e-03, 4.4564e-02, 5.7512e-03, 9.3530e-04,
         1.4572e-02, 6.2075e-03, 3.1225e-03],
        [2.4987e-01, 2.4486e-02, 2.8669e-01, 1.5228e-02, 2.8864e-02, 4.3187e-02,
         5.5294e-02, 8.2267e-02, 2.1411e-01],
        [7.1092e-02, 8.7373e-02, 4.1576e-01, 2.2153e-01, 8.9381e-03, 1.1020e-02,
         3.6452e-02, 2.6599e-02, 1.2124e-01],
        [2.0167e-01, 1.2511e-01, 1.4100e-01, 4.7089e-02, 4.2112e-02, 1.3234e-01,
         9.5376e-02, 1.6339e-01, 5.1919e-02],
        [2.1234e-01, 1.6746e-02, 1.0519e-01, 1.7828e-02, 6.2450e-02, 2.4318e-01,
         5.8937e-02, 1.3822e-01, 1.4510e-01],
        [2.9630e-01, 3.4920e-02, 5.3029e-02, 1.3975e-01, 1.0845e-02, 1.8033e-02,
         9.1013e-02, 8.4441e-0

# Visualize model

In [24]:
!pip3 install bertviz

     |████████████████████████████████| 163kB 4.4MB/s 
     |████████████████████████████████| 133kB 6.9MB/s 
     |████████████████████████████████| 1.2MB 5.7MB/s 
     |████████████████████████████████| 7.4MB 16.4MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
ERROR: botocore 1.20.53 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [25]:
from bertviz import model_view

In [26]:
def show_model_view(model, tokenizer, sentence_a, sentence_b=None, hide_delimiter_attn=False, display_mode="dark"):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        attention = model(input_ids, token_type_ids=token_type_ids, output_attentions=True).attentions
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        attention = model(input_ids)[-1]
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)  
    if hide_delimiter_attn:
        for i, t in enumerate(tokens):
            if t in ("[SEP]", "[CLS]"):
                for layer_attn in attention:
                    layer_attn[0, :, i, :] = 0
                    layer_attn[0, :, :, i] = 0
    model_view(attention, tokens, sentence_b_start, display_mode=display_mode)

In [29]:
sentence_a = "The cat sat on the mat"
sentence_b = "The cat lay on the rug"
show_model_view(model, tokenizer, sentence_a, sentence_b, hide_delimiter_attn=False, display_mode="light")

Output hidden; open in https://colab.research.google.com to view.